In [1]:
# Usual Modules
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
from ase.build import molecule
from ase.calculators.vasp import Vasp
import matplotlib.pyplot as plt

# GA specific Modules
from GA_runner import *
from ase.db import connect
from ase.ga.offspring_creator import OperationSelector
from ase.ga.standardmutations import RattleMutation, RotationalMutation, RattleRotationalMutation
from ase.ga.data import PrepareDB, DataConnection
from ase.ga.standard_comparators import InteratomicDistanceComparator
from ase.ga.population import Population



In [2]:
# Structures to evaluate GA
STRUCT = read('./CHA_CONTCAR')
ADSORB = read('./TMADA_CONTCAR') 

# Variables needed before running GA

init_pop_size = 20   # initial structures generated (this number will need to be high, it will be about 1/5 the size once we eliminate bad structures)
Ads_cent =[2.,-5,5] # center adsorbate at some point in the cage

n_to_test = 10 # number of children to make in each generation
mult = 1.4 # factor to multiply covalent radii

db_file = 'CHA_TMADA.db' # database name

In [3]:
zeoGA = GenerateParent(STRUCT,ADSORB,init_pop_size) # initialize GA
structs = zeoGA.construct_parent(Ads_cent) # Create parents centered at a point (is none supplied it is centerd at 0,0,0)

# Get a list of atoms to optimize in the GA
chem_sym = ADSORB.get_chemical_symbols()
elem = np.unique(chem_sym)
atom_numbers = [None]
for i in elem:
    count = chem_sym.count(i)
    if i == 'C':
        atom_num = 6
    elif i =='H':
        atom_num = 1
    atom_numbers = count * [atom_num] + atom_numbers
    
del atom_numbers[-1]

In [ ]:
view(structs)

In [4]:
# prepare DB

my_file = Path('./{0}'.format(db_file))
if my_file.is_file():
    print('{0} already exists. Please delete file and rerun. No structures have been added to the DB'.format(db_file))

else:
    d = PrepareDB(db_file_name= db_file,
             simulation_cell = STRUCT,
             stoichiometry = atom_numbers)

    # add parents to database

    for i in structs:
        d.add_unrelaxed_candidate(i)
        
    print('DB has been created and updated with parents')

DB has been created and updated with parents


In [5]:
# set up variables for GA (should be consistent with variables above or defaults are fine)
da = DataConnection(db_file)
atom_numbers_to_optimize = da.get_atom_numbers_to_optimize()
n_to_optimize = len(atom_numbers_to_optimize)
frame = da.get_slab()
all_atom_types = get_all_atom_types(frame, atom_numbers_to_optimize)
blmin = closest_distances_generator(all_atom_types,ratio_of_covalent_radii=mult)

comp = InteratomicDistanceComparator(n_top = n_to_optimize,
                                    pair_cor_cum_diff=0.015,  # these are traditional defaults
                                    pair_cor_max=0.7,
                                    dE=0.02,
                                    mic=False)

oclist = [(1.,RattleMutation(blmin,n_to_optimize,use_tags=True)),   # These mutations have been selected for our problem and can change
          (1.,RotationalMutation(blmin,n_to_optimize)),
          (1.,RattleRotationalMutation(RattleMutation(blmin,n_to_optimize,use_tags=True),
                                       RotationalMutation(blmin,n_to_optimize)))]

opselect = OperationSelector(*zip(*oclist))

In [ ]:
# Write calculations to Folder Candidates (the calculations are not running. Must be performed with ase.calculators.vasp)
for i in range(2,da.get_number_of_unrelaxed_candidates()+2):
    atoms = da.get_atoms(id=i)
    calc = Vasp(directory = './Candidates/Can-{:02d}'.format(i),
                xc = 'pbe', 
                encut = 520,                                
                ediff = 1e-5,
                ediffg = -0.01, 
                ibrion = 2,
                isif = 2, 
                nsw = 600,
                ispin = 1,
                nelm = 60,
                sigma = 0.01,
                ismear = 0,
                lreal='A',
                ncore = 24,                                   # set ncore depending on number of cores
                ivdw = 12,
                algo = 'VeryFast',
                lcharg = False,
                lwave = False)
    
    calc.write_input(atoms)

In [6]:
# update database
num = UpdateDB(db_file).update()

Updating candidate 2
Updating candidate 3
Updating candidate 4
Updating candidate 5
Updating candidate 6
Updating candidate 7
Updating candidate 8
Updating candidate 9
Updating candidate 10
Updating candidate 11
Updating candidate 12
Updating candidate 13
Updating candidate 14
Updating candidate 15
Updating candidate 16
Updating candidate 17
Updating candidate 18
Updating candidate 19
Updating candidate 20
Updating candidate 21


In [7]:
# create the population
da = DataConnection(db_file)
population = Population(data_connection=da,
                        population_size=init_pop_size,
                        comparator=comp)

# generate children
zeochild1 = GenerateChild(db_file, population, opselect, n_to_test, num)
Operation, Marraige = zeochild1.construct_child()

print(Marraige,Operation)

Now starting configuration number 22
Now starting configuration number 23
Now starting configuration number 24
Now starting configuration number 25
Now starting configuration number 26
Now starting configuration number 27
Now starting configuration number 28
Now starting configuration number 29
Now starting configuration number 30
Now starting configuration number 31
[[18, 10], [11, 18], [18, 20], [18, 12], [20, 18], [18, 20], [20, 14], [20, 4], [12, 20], [14, 12]] ['mutation: rotational', 'mutation: rotational', 'mutation: rattlerotational', 'mutation: rattlerotational', 'mutation: rattle', 'mutation: rattle', 'mutation: rotational', 'mutation: rattle', 'mutation: rattlerotational', 'mutation: rattle']


In [8]:
# Run first generationof children 
da = DataConnection(db_file)
for i in range(num+1,num+n_to_test+1):
    atoms = da.get_atoms(id=i)
    calc = Vasp(directory = './Candidates/Can-{:02d}'.format(i),
                xc = 'pbe', 
                encut = 520,                                
                ediff = 1e-5,
                ediffg = -0.01, 
                ibrion = 2,
                isif = 2, 
                nsw = 600,
                ispin = 1,
                nelm = 60,
                sigma = 0.01,
                ismear = 0,
                lreal='A',
                ncore = 24,                                   # set ncore depending on number of cores
                ivdw = 12,
                algo = 'VeryFast',
                lcharg = False,
                lwave = False)
        
    calc.write_input(atoms)

In [9]:
# update database
num = UpdateDB(db_file).update()

Updating candidate 22
Updating candidate 23
Updating candidate 24
Updating candidate 25
Updating candidate 26
Updating candidate 27
Updating candidate 28
Updating candidate 29
Updating candidate 30
Updating candidate 31


In [10]:
# Recreate population
da = DataConnection(db_file)
population = Population(data_connection=da,
                        population_size=init_pop_size+n_to_test,
                        comparator=comp)

# generate children
zeochild2 = GenerateChild(db_file, population, opselect, n_to_test, num)
Operation, Marraige = zeochild2.construct_child()

print(Marraige,Operation)

Now starting configuration number 32
Now starting configuration number 33
Now starting configuration number 34
Now starting configuration number 35
Now starting configuration number 36
Now starting configuration number 37
Now starting configuration number 38
Now starting configuration number 39
Now starting configuration number 40
Now starting configuration number 41
[[14, 18], [20, 11], [10, 11], [11, 18], [23, 4], [18, 4], [20, 18], [11, 18], [14, 18], [14, 4]] ['mutation: rotational', 'mutation: rattlerotational', 'mutation: rattlerotational', 'mutation: rotational', 'mutation: rattle', 'mutation: rotational', 'mutation: rattlerotational', 'mutation: rattle', 'mutation: rotational', 'mutation: rotational']


In [11]:
# Run first generationof children 
for i in range(num+1,num+n_to_test+1):
    atoms = da.get_atoms(id=i)
    calc = Vasp(directory = './Candidates/Can-{:02d}'.format(i),
                xc = 'pbe', 
                encut = 520,                                
                ediff = 1e-5,
                ediffg = -0.01, 
                ibrion = 2,
                isif = 2, 
                nsw = 600,
                ispin = 1,
                nelm = 60,
                sigma = 0.01,
                ismear = 0,
                lreal='A',
                ncore = 24,                                   # set ncore depending on number of cores
                ivdw = 12,
                algo = 'VeryFast',
                lcharg = False,
                lwave = False)
    
    calc.write_input(atoms)

In [ ]:
# update database for final time
num = UpdateDB(db_file).update()